# tsam - Optimal combination of segments and periods 
Determine the optimal combination of segments and periods during the aggregation of the time series
Date: 27.04.2022

Author: Leander Kotzur

Import pandas and the relevant time series aggregation class

In [ ]:
%load_ext autoreload
%autoreload 2

import os

import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import tsam
import tsam.hyperparametertuning as tune
from tsam.timeseriesaggregation import TimeSeriesAggregation

### Input data 

Read in time series from testdata.csv with pandas

In [ ]:
raw = pd.read_csv("testdata.csv", index_col=0)
raw = raw.rename(
    columns={
        "T": "Temperature [°C]",
        "Load": "Load [kW]",
        "Wind": "Wind [m/s]",
        "GHI": "Solar [W/m²]",
    }
)

Create a plot function for a visual comparison of the time series

In [ ]:
def plotTS_all(plot_data, raw_data, period_hours=24):
    """Create heatmaps for all columns using plotly."""
    columns = list(raw_data.columns)
    n_cols = len(columns)

    fig = make_subplots(rows=n_cols, cols=1, subplot_titles=columns)

    for i, col in enumerate(columns, 1):
        stacked, _ = tsam.unstackToPeriods(plot_data[[col]].copy(), period_hours)

        fig.add_trace(
            go.Heatmap(
                z=stacked[col].values.T,
                colorscale="Viridis",
                zmin=raw_data[col].min(),
                zmax=raw_data[col].max(),
                colorbar={
                    "title": col,
                    "y": 1 - (i - 0.5) / n_cols,
                    "len": 0.9 / n_cols,
                },
            ),
            row=i,
            col=1,
        )
        fig.update_yaxes(title_text="Hour", row=i, col=1)

    fig.update_xaxes(title_text="Day", row=n_cols, col=1)
    fig.update_layout(height=200 * n_cols, title="Time Series Heatmaps")
    return fig

Plot an example series - in this case the temperature

In [ ]:
# Original data heatmaps
plotTS_all(raw, raw, period_hours=24)

### Tune a hierarchical aggregation with segments in combination with duration representation

In [ ]:
tunedAggregations = tune.HyperTunedAggregations(
    TimeSeriesAggregation(
        raw,
        hoursPerPeriod=24,
        clusterMethod="hierarchical",
        representationMethod="durationRepresentation",
        distributionPeriodWise=False,
        rescaleClusterPeriods=False,
        segmentation=True,
    )
)

And determine the pareto optimal aggregation up to 200 total time steps. This may take some time...

In [ ]:
tunedAggregations.identifyParetoOptimalAggregation(untilTotalTimeSteps=8760)

  1%|▏         | 130/8760 [05:38<6:47:31,  2.83s/it]

And show the results for the last aggregation

In [ ]:
predictedPeriods = tunedAggregations.aggregationHistory[-1].predictOriginalData()

In [ ]:
# Reconstructed data heatmaps
plotTS_all(
    predictedPeriods,
    raw,
    period_hours=tunedAggregations.base_aggregation.hoursPerPeriod,
)

Save the resulting combination

In [ ]:
pd.DataFrame(
    {
        "segments": tunedAggregations._segmentHistory,
        "periods": tunedAggregations._periodHistory,
    }
).to_csv(os.path.join("results", "paretoOptimalAggregation.csv"))